In [ ]:
# Train api
# pytorch 직접구현

In [ ]:
# 전처리
from transformers import BertTokenizer
BERT_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_NAME)

In [ ]:
# 데이터 준비
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('movie_reviews')
ids = movie_reviews.fileids()
reviews = [movie_reviews.raw(id) for id in ids]
categoris = [movie_reviews.categories(id)[0] for id in ids]

In [ ]:
# 라벨인코딩
y = [ 1 if c=='pos' else 0 for c in categoris]

In [ ]:
# train/test split
x_train,x_test,y_train,y_test = train_test_split(reviews,y,stratify=y,random_state=42,test_size=0.2,shuffle=True)

In [ ]:
# 데이터셋
import torch
from torch.utils.data import Dataset, DataLoader
class MovieReivewDataset(Dataset):
    '''
    Args
        encodings : 토크나이져된 값(딕셔너리형태)
        labels : 라벨링된 클래스값(0 1)
    '''
    def __init__(self, encodings, labels):        
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, index):  # torch 데이터를받는다
        item = {
            key:val[index].clone().detach()
            for key,val in self.encodings.items()
        }
        item['labels'] = torch.tensor(self.labels[index], dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)

# 토큰화  (BertTokenizerFast)  --- huggingFace 공식 추천
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
train_encodings =  tokenizer(
    x_train,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)
test_encodings =  tokenizer(
    x_test,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)
train_dataset = MovieReivewDataset(train_encodings,y_train)
test_dataset = MovieReivewDataset(test_encodings,y_test)        

In [ ]:
# Trainer API
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import evaluate  # load_metric
# 모델 로드  클래스개수 2
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy' : accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
        'precision' : precision_metric.compute(predictions=predictions, references=labels)['precision'],
        'recall' : recall_metric.compute(predictions=predictions, references=labels)['recall'],
        'f1' : f1_metric.compute(predictions=predictions, references=labels)['f1'],
    }

# TraingAguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,             # NLP 에서는 2 ~5
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=16, # 평가는 gradient 안하므로 train보다 크게설정하는 경향
    # learning_rate=2e-5,
    # weight_decay=0.01,  # L2정규화 규제 강도
    # logging_dir='./logs',
    # logging_steps=50,
    # eval_strategy='epoch',
    # save_strategy='epoch',
    # load_best_model_at_end=True,
    # metric_for_best_model='f1',
    report_to='none'
)
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics
)
# 학습진행
trainer.train()

In [ ]:
# Pytorch로 직접개선
# 메모리 정리
del model, trainer
torch.cuda.empty_cache()

# BertForSequenceClassification --> Bert전용모델
# 분류용 헤드를 포함 : [CLS]토큰을 출력 -> Linear layer -> logits

# BertModel : Bert 전용
# 분류헤드없음 -> 분류 회귀 QA등 작업에 바로 쓰려면 헤드에 붙여야한다.
from transformers import BertModel
import torch.nn.functional as F
from torch.optim import AdamW
import time